# Processing of SCIP features

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../common_.py
import sys
sys.path.append("..")
from common import *

In [3]:
import pyarrow.parquet as pq
import pyarrow
from pandas.api.types import CategoricalDtype

## SCIP computed features

In [5]:
data_dir = Path("/home/maximl/scratch/data/CD7/")

In [7]:
data_dir = Path("/data/gent/vo/000/gvo00070/vsc42015/datasets/cd7/800/results/scip/202203211633/")

In [8]:
df = pandas.concat([
    pq.read_table(p).to_pandas()
    for p in Path(data_dir / "").glob("*.parquet")
], axis=0)
df = df.reset_index(drop=True)

In [9]:
df.shape

(124409, 1494)

In [10]:
df["meta_path"] = df["meta_path"].map(lambda a: "/data/gent/vo/000/gvo00070/vsc42015/datasets/cd7/800/" + os.path.basename(a))

In [11]:
df["meta_P"] = df["meta_scene"].apply(lambda s: int(s.split("-")[0][1:]))
df["meta_replicate"] = df["meta_scene"].apply(lambda s: int(s.split("-")[1][1:]))
df["meta_panel"] = df["meta_scene"].apply(lambda s: s.split("-")[1][0])

In [12]:
cat_type = CategoricalDtype(categories=sorted(df["meta_P"].unique()), ordered=True)
df["meta_P"] = df["meta_P"].astype(cat_type)
cat_type = CategoricalDtype(categories=sorted(df["meta_replicate"].unique()), ordered=True)
df["meta_replicate"] = df["meta_replicate"].astype(cat_type)
cat_type = CategoricalDtype(categories=sorted(df["meta_panel"].unique()), ordered=True)
df["meta_panel"] = df["meta_panel"].astype(cat_type)

In [13]:
def add_id(df):
    df.insert(0, "meta_id", numpy.arange(len(df)))
    return df
df = df.groupby(["meta_panel", "meta_replicate", "meta_P"]).apply(lambda df: add_id(df))

In [16]:
pq.write_table(pyarrow.Table.from_pandas(df), data_dir / "features.parquet")